In [1]:
import numpy as np
from root_numpy import root2array, tree2array, rec2array
import pandas as pd

branch_names = """momentumx, momentumy,momentumz,momentum,energy,position,MvdDEDX,MvdHits,SttMeanDEDX,SttHits,GemHits,TofStopTime,
TofM2,TofTrackLength,TofQuality,TofBeta,DrcThetaC,DrcQuality,
DiscThetaC,DiscQuality,
EmcRawEnergy,EmcCalEnergy,EmcQuality,EmcNumberOfCrystals,EmcNumberOfBumps,EmcModule,
EmcZ20,EmcZ53,EmcLat,EmcE1,EmcE9,EmcE25,MuoQuality,MuoIron,MuoMomentumIn,MuoNumberOfLayers,MuoModule,MuoHits,
DegreesOfFreedom,ChiSquared""".split(",")

branch_names = [c.strip() for c in branch_names]
branch_names = list(branch_names)

electrons = root2array("/home/ikp1/esmail/Files/treeElectrons.root", "t1", branch_names)
electrons = rec2array(electrons)

pions = root2array("/home/ikp1/esmail/Files/treePions.root", "t1", branch_names)
pions = rec2array(pions)

muons = root2array("/home/ikp1/esmail/Files/treeMuons.root", "t1", branch_names)
muons = rec2array(muons)

kaons = root2array("/home/ikp1/esmail/Files/treeKaons.root", "t1", branch_names)
kaons = rec2array(kaons)

anti_p = root2array("/home/ikp1/esmail/Files/treeProtons.root", "t1", branch_names)
anti_p = rec2array(anti_p)

X = np.concatenate((electrons, pions, muons, kaons, anti_p))
y = np.concatenate(( np.zeros(electrons.shape[0]),np.ones(pions.shape[0]), (2*np.ones(muons.shape[0])), (3*np.ones(kaons.shape[0])), (4*np.ones(anti_p.shape[0])) ))
df = pd.DataFrame(np.hstack((X, y.reshape(y.shape[0], -1))),columns=branch_names+['temp'])

df['Momentum'] = np.sqrt(df.loc[:,'momentum']) 
df = df.drop(['momentum'], axis=1)
df['E9/E25'] = df.loc[:,'EmcE9']/df.loc[:,'EmcE25']
df['E1/E25'] = df.loc[:,'EmcE1']/df.loc[:,'EmcE25']
df['E/p'] = df.loc[:,'EmcCalEnergy']/df.loc[:,'Momentum']
df['labels'] = df.loc[:,'temp']
df = df.drop(['temp'], axis=1)
df = df.dropna()

%matplotlib tk

In [83]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import keras
from keras import optimizers
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adam, Adagrad
from keras.callbacks import EarlyStopping, ModelCheckpoint

X_train,X_test, y_train,y_test = train_test_split(df.iloc[:,0:-1], df.iloc[:,-1], test_size=0.30, random_state=0)

# scale the features
from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.fit_transform(X_test)


# compute the mean value per feature on the training set
mean_on_train = X_train.mean(axis=0)
# compute the standard deviation of each feature on the training set
std_on_train = X_train.std(axis=0)
# subtract the mean, scale by inverse standard deviation
# afterwards, mean=0 and std=1
X_train_scaled = (X_train - mean_on_train) / std_on_train
# use THE SAME transformation (using training mean and std) on the test set
X_test_scaled = (X_test - mean_on_train) / std_on_train



batch_size = 512 #128, 256
num_classes = 5
epochs = 500

#print(y_train.shape)
# convert class vectors to binary class matrices
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

encoder = LabelEncoder()
encoder.fit(y_train)
encoded_y_train = encoder.transform(y_train)
encoder.fit(y_test)
encoded_y_test = encoder.transform(y_test)

from keras.utils.np_utils import to_categorical
one_hot_train_labels = to_categorical(encoded_y_train)
one_hot_test_labels = to_categorical(encoded_y_test)
print(encoded_y_train.shape)
print(encoded_y_train.shape)

model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(43,)))
#model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(36, activation='relu'))
#model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))

model.summary()
model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

history = model.fit(X_train_scaled , y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks=[EarlyStopping(verbose=True, patience=20, monitor='val_loss')],
                   validation_data=(X_test_scaled , y_test))
score = model.evaluate(X_test_scaled, y_test, verbose=0)
print('Test loss:%.2f' %score[0])
print('Test accuracy:%.2f' % score[1])

(109487,)
(109487,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_159 (Dense)            (None, 32)                1408      
_________________________________________________________________
dense_160 (Dense)            (None, 32)                1056      
_________________________________________________________________
dense_161 (Dense)            (None, 5)                 165       
Total params: 2,629
Trainable params: 2,629
Non-trainable params: 0
_________________________________________________________________
Train on 109487 samples, validate on 46924 samples
Epoch 1/500
109487/109487 [==============================] - 2s 21us/step - loss: 1.1112 - acc: 0.5218 - val_loss: 0.9346 - val_acc: 0.5914
Epoch 2/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.8609 - acc: 0.6244 - val_loss: 0.8131 - val_acc: 0.6474
Epoch 3/500
109487/109487 [===========================

Epoch 54/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3957 - acc: 0.8503 - val_loss: 0.4212 - val_acc: 0.8420
Epoch 55/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3941 - acc: 0.8516 - val_loss: 0.4160 - val_acc: 0.8438
Epoch 56/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3927 - acc: 0.8515 - val_loss: 0.4122 - val_acc: 0.8468
Epoch 57/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3912 - acc: 0.8517 - val_loss: 0.4112 - val_acc: 0.8475
Epoch 58/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3888 - acc: 0.8540 - val_loss: 0.4115 - val_acc: 0.8469
Epoch 59/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3887 - acc: 0.8539 - val_loss: 0.4109 - val_acc: 0.8455
Epoch 60/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3873 - acc: 0.8535 - val_loss: 0.4067 - val_acc: 0.8485
Epoch 61/500
109487/

109487/109487 [==============================] - 1s 5us/step - loss: 0.3441 - acc: 0.8705 - val_loss: 0.3694 - val_acc: 0.8631
Epoch 113/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3455 - acc: 0.8701 - val_loss: 0.3696 - val_acc: 0.8619
Epoch 114/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3438 - acc: 0.8701 - val_loss: 0.3670 - val_acc: 0.8633
Epoch 115/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3439 - acc: 0.8704 - val_loss: 0.3669 - val_acc: 0.8641
Epoch 116/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3432 - acc: 0.8705 - val_loss: 0.3675 - val_acc: 0.8632
Epoch 117/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3430 - acc: 0.8703 - val_loss: 0.3681 - val_acc: 0.8633
Epoch 118/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3446 - acc: 0.8698 - val_loss: 0.3696 - val_acc: 0.8618
Epoch 119/500
109487/109487

109487/109487 [==============================] - 1s 5us/step - loss: 0.3292 - acc: 0.8750 - val_loss: 0.3563 - val_acc: 0.8648
Epoch 171/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3284 - acc: 0.8747 - val_loss: 0.3557 - val_acc: 0.8667
Epoch 172/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3287 - acc: 0.8754 - val_loss: 0.3555 - val_acc: 0.8660
Epoch 173/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3278 - acc: 0.8752 - val_loss: 0.3581 - val_acc: 0.8644
Epoch 174/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3284 - acc: 0.8749 - val_loss: 0.3542 - val_acc: 0.8665
Epoch 175/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3281 - acc: 0.8749 - val_loss: 0.3546 - val_acc: 0.8669
Epoch 176/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3283 - acc: 0.8748 - val_loss: 0.3539 - val_acc: 0.8679
Epoch 177/500
109487/109487

109487/109487 [==============================] - 1s 5us/step - loss: 0.3153 - acc: 0.8789 - val_loss: 0.3429 - val_acc: 0.8693
Epoch 229/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3146 - acc: 0.8792 - val_loss: 0.3392 - val_acc: 0.8707
Epoch 230/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3151 - acc: 0.8788 - val_loss: 0.3402 - val_acc: 0.8702
Epoch 231/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3147 - acc: 0.8787 - val_loss: 0.3394 - val_acc: 0.8717
Epoch 232/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3140 - acc: 0.8788 - val_loss: 0.3396 - val_acc: 0.8697
Epoch 233/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3127 - acc: 0.8796 - val_loss: 0.3411 - val_acc: 0.8693
Epoch 234/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3138 - acc: 0.8787 - val_loss: 0.3415 - val_acc: 0.8703
Epoch 235/500
109487/109487

109487/109487 [==============================] - 1s 5us/step - loss: 0.3050 - acc: 0.8813 - val_loss: 0.3299 - val_acc: 0.8744
Epoch 287/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3029 - acc: 0.8827 - val_loss: 0.3319 - val_acc: 0.8720
Epoch 288/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3036 - acc: 0.8823 - val_loss: 0.3313 - val_acc: 0.8735
Epoch 289/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3032 - acc: 0.8825 - val_loss: 0.3288 - val_acc: 0.8748
Epoch 290/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3027 - acc: 0.8827 - val_loss: 0.3294 - val_acc: 0.8755
Epoch 291/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3027 - acc: 0.8835 - val_loss: 0.3301 - val_acc: 0.8742
Epoch 292/500
109487/109487 [==============================] - 1s 5us/step - loss: 0.3048 - acc: 0.8824 - val_loss: 0.3302 - val_acc: 0.8741
Epoch 293/500
109487/109487

In [84]:
print ('Testing...')
yhat = model.predict(X_test_scaled, verbose = True, batch_size = 512)
print(yhat)
import matplotlib.pyplot as plt
%matplotlib tk
ann = history.history
plt.grid()
plt.plot(100 * np.array(ann['acc']), label='training')
plt.plot(100 * np.array(ann['val_acc']), label='validation')
plt.xlim(0)
plt.xlabel('Iterations', fontsize=20)
plt.ylabel('Accuracy %', fontsize=20)
plt.legend(loc='lower right', fontsize=20)


Testing...
46924/46924 [==============================] - 1s 12us/step
[[1.08e-03 8.02e-01 1.91e-01 6.71e-03 3.92e-06]
 [5.10e-03 7.17e-02 9.23e-01 3.78e-09 8.03e-09]
 [2.02e-16 2.99e-06 7.26e-30 1.57e-03 9.98e-01]
 ...
 [1.86e-07 8.40e-01 1.51e-01 9.54e-03 1.05e-04]
 [1.46e-09 2.02e-03 3.03e-17 1.10e-01 8.88e-01]
 [1.61e-09 4.55e-03 9.85e-06 9.95e-01 4.76e-04]]


In [86]:
yhat_cls = np.argmax(yhat, axis=1)
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, yhat_cls)
np.set_printoptions(precision=2)
plot_confusion_matrix(cnf_matrix, classes=['e','pi','mu','ka','p'],
                      normalize=True,
                      title='Normalized confusion matrix')

Normalized confusion matrix
[[0.98 0.01 0.   0.   0.01]
 [0.01 0.78 0.09 0.08 0.03]
 [0.   0.06 0.92 0.01 0.  ]
 [0.01 0.09 0.02 0.79 0.08]
 [0.01 0.03 0.01 0.06 0.9 ]]
